In [ ]:
library(SNPRelate)

# Format conversion

SNPRelate requires gds, so convert the vcf to GDS format

In [ ]:
system("rm -f flowers.gds")
snpgdsVCF2GDS("bwa_msdr_MR_ih_lc_nr503_F.vcf.gz", "flowers.gds",
              ignore.chr.prefix = c("scaffold_", "chromosome_"))

# PCA

Flowers et al. state they used SNPrelate to perform PCA decomposition. Here we use default parameters on Flower's filtered VCF.

In [ ]:
snpgdsSummary("flowers.gds")

In [ ]:
geno <- snpgdsOpen("flowers.gds")

In [ ]:
pca <- snpgdsPCA(geno, num.thread=4, verbose = T)

# Plot

In [ ]:
library(ggplot2)

The names of lines in the VCF do not match what is given in the SRA database. Our metadata table (from the SRA) has line IDs like CC-1010, whereas the VCF has CR1010. The below converts VCF names to SRA names.

In [ ]:
sra_names = sub("CR", "CC-", plotdat$sample)

Import metadata from the kWIP analysis under 'writeups'

In [ ]:
chlamy_meta = read.delim("../writeups/chlamy/chlamy_meta.tab")

Note that all the "sra names" from above conversion match the names in the SRA metadata

In [ ]:
m = match(sra_names, chlamy_meta$strain)
m

Reorder the metadata, assert the names match

In [ ]:
chlamy_meta = chlamy_meta[m, ]
print(paste(chlamy_meta$strain, sra_names))

Assemble all data & metadata for plotting

In [ ]:
plotdat = data.frame(sample=pca$sample.id,
                     sraname=sra_names,
                     region=chlamy_meta$origin,
                     mbases=chlamy_meta$MBases,
                     PC1=pca$eigenvect[,1],
                     PC2=pca$eigenvect[,2],
                     PC3=pca$eigenvect[,3])

In [ ]:
ggplot(plotdat, aes(x=PC1, y=PC2)) +
    geom_point(aes(colour=region)) +
    theme_bw()

The above plot is upside-down from the flowers et al. plot. Reverse PC2 and try again

In [ ]:
plotdat$PC2 = -plotdat$PC2

### Proper plot

In [ ]:
cols = c("light blue", "blue", "dark green", "red" )
p = ggplot(plotdat, aes(x=PC1, y=PC2)) +
    geom_point(aes(colour=region), size=3) +
    scale_color_manual(values = cols) +
    theme_classic() +
    theme(panel.border=element_rect(colour = "black", fill=NA))

print(p)

pdf("chlamy.pdf", width=3.5, height=3.5)
print(p)
dev.off()

In [ ]:
geno